In [2]:
import tensorflow as tf
tf.__version__


'2.12.0'

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [4]:
subject = 'Espírito Santo - Consumo de Cimento (t)'
split_index = 6 #Referente aos 7 anos de input  

In [5]:
data = pd.read_csv('../Inputs Atuais/2014_01_model_input_ES.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data

,Ano,Espírito Santo - IDH,Espírito Santo - PIB - Estadual,Espírito Santo - PIB - Construção Civil,Espírito Santo - PIB - Per Capita,Espírito Santo - PIB - Preços de Mercado,Espírito Santo - IDH Educacao,Espírito Santo - IDH Longevidade,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,Espírito Santo - IDH Renda,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Espírito Santo - Produção de Cimento (t),Espírito Santo - Consumo de Cimento (t)
0,2014,0.773701,7.218048e+07,3.534361e+06,16.602277,6.008490e+07,0.726354,0.866497,5.676413e+08,2.779340e+09,...,0.737011,20.943935,1016.338200,1416.529798,24.555390,19.517780,75.430446,1.825872,2184.851676,1296.041857
1,2015,0.778964,7.217382e+07,3.504012e+06,16.477049,5.965484e+07,0.735243,0.880249,5.179392e+08,2.561731e+09,...,0.730914,20.016361,1016.401227,1398.065883,24.641173,19.598871,75.258234,1.793707,1939.379336,1250.323886
2,2016,0.779234,7.236241e+07,3.485388e+06,16.442296,5.953784e+07,0.736083,0.880090,4.524144e+08,2.516859e+09,...,0.730906,19.950370,1016.507582,1365.698492,24.390776,19.578235,75.629199,1.728129,1632.640239,1210.917257
3,2017,0.779255,7.274917e+07,3.492937e+06,16.497126,5.973414e+07,0.736454,0.879462,4.213689e+08,2.497710e+09,...,0.731261,20.563357,1016.418014,1341.552229,24.409722,19.738208,76.145059,1.661760,1275.568674,1097.369000
4,2018,0.779188,7.296296e+07,3.509673e+06,16.563003,5.993212e+07,0.736950,0.878322,4.166580e+08,2.797145e+09,...,0.731648,21.207610,1016.292908,1319.915562,24.585881,19.820567,76.203440,1.625136,995.354714,1146.390000
5,2019,0.778386,7.233667e+07,3.535172e+06,16.533154,5.977718e+07,0.736400,0.876624,4.462625e+08,3.484252e+09,...,0.731343,20.815964,1016.221265,1317.440738,24.601436,19.646717,75.523951,1.619791,869.405656,1139.906000
6,2020,0.777058,7.157970e+07,3.549612e+06,16.538992,5.971385e+07,0.735982,0.874317,5.489412e+08,4.114911e+09,...,0.729920,19.775385,1016.081947,1291.459808,24.435049,19.419591,75.362851,1.595958,826.251278,1138.655000
7,2021,0.775700,7.062588e+07,3.555015e+06,16.597316,5.977368e+07,0.733980,0.872437,6.638811e+08,4.421848e+09,...,0.729640,18.983426,1016.033139,1232.023456,24.175917,19.208908,75.604121,1.563901,810.222414,1135.022000
8,2022,0.776127,6.945073e+07,3.562707e+06,16.663260,5.982392e+07,0.733251,0.873204,7.010332e+08,4.533216e+09,...,0.730971,18.283852,1016.062498,1209.222772,24.107790,19.095486,75.405291,1.561888,803.434154,1040.260000


In [6]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Espírito Santo - IDH,Espírito Santo - PIB - Estadual,Espírito Santo - PIB - Construção Civil,Espírito Santo - PIB - Per Capita,Espírito Santo - PIB - Preços de Mercado,Espírito Santo - IDH Educacao,Espírito Santo - IDH Longevidade,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Espírito Santo - Desemprego,Espírito Santo - IDH Renda,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Espírito Santo - Produção de Cimento (t)
0,0.773701,7.218048e+07,3.534361e+06,16.602277,6.008490e+07,0.726354,0.866497,5.676413e+08,2.779340e+09,1.954361e+10,...,10.450155,0.737011,20.943935,1016.338200,1416.529798,24.555390,19.517780,75.430446,1.825872,2184.851676
1,0.778964,7.217382e+07,3.504012e+06,16.477049,5.965484e+07,0.735243,0.880249,5.179392e+08,2.561731e+09,1.327596e+10,...,10.942052,0.730914,20.016361,1016.401227,1398.065883,24.641173,19.598871,75.258234,1.793707,1939.379336
2,0.779234,7.236241e+07,3.485388e+06,16.442296,5.953784e+07,0.736083,0.880090,4.524144e+08,2.516859e+09,1.011370e+10,...,10.979229,0.730906,19.950370,1016.507582,1365.698492,24.390776,19.578235,75.629199,1.728129,1632.640239
3,0.779255,7.274917e+07,3.492937e+06,16.497126,5.973414e+07,0.736454,0.879462,4.213689e+08,2.497710e+09,8.456309e+09,...,10.448735,0.731261,20.563357,1016.418014,1341.552229,24.409722,19.738208,76.145059,1.661760,1275.568674
4,0.779188,7.296296e+07,3.509673e+06,16.563003,5.993212e+07,0.736950,0.878322,4.166580e+08,2.797145e+09,7.937760e+09,...,10.241458,0.731648,21.207610,1016.292908,1319.915562,24.585881,19.820567,76.203440,1.625136,995.354714
5,0.778386,7.233667e+07,3.535172e+06,16.533154,5.977718e+07,0.736400,0.876624,4.462625e+08,3.484252e+09,1.091282e+10,...,10.168769,0.731343,20.815964,1016.221265,1317.440738,24.601436,19.646717,75.523951,1.619791,869.405656
6,0.777058,7.157970e+07,3.549612e+06,16.538992,5.971385e+07,0.735982,0.874317,5.489412e+08,4.114911e+09,1.625333e+10,...,10.043678,0.729920,19.775385,1016.081947,1291.459808,24.435049,19.419591,75.362851,1.595958,826.251278


In [7]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0    1250.323886
1    1210.917257
2    1097.369000
3    1146.390000
4    1139.906000
5    1138.655000
6    1135.022000
7    1040.260000
8            NaN
Name: Espírito Santo - Consumo de Cimento (t), dtype: float64

In [8]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Espírito Santo - IDH,Espírito Santo - PIB - Estadual,Espírito Santo - PIB - Construção Civil,Espírito Santo - PIB - Per Capita,Espírito Santo - PIB - Preços de Mercado,Espírito Santo - IDH Educacao,Espírito Santo - IDH Longevidade,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Espírito Santo - Desemprego,Espírito Santo - IDH Renda,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Espírito Santo - Produção de Cimento (t)
0,-2.211091,-0.951362,1.270512,1.552867,1.665172,-2.214447,-2.161616,1.779974,0.019134,2.003355,...,-0.279545,2.220553,0.776643,-0.271131,1.511196,0.257387,-1.299945,-0.755174,1.459630,1.463425
1,0.421684,-0.973968,-0.329186,-0.786497,-0.737408,0.178233,0.703083,0.870373,-0.621387,0.401153,...,1.278707,-0.582352,-1.218894,0.412431,1.018762,1.152736,-0.503063,-1.240539,1.057699,0.951690
2,0.556495,-0.334188,-1.310849,-1.435708,-1.391070,0.404405,0.669927,-0.328802,-0.753466,-0.407217,...,1.396478,-0.585964,-1.360865,1.565924,0.155523,-1.460752,-0.705856,-0.195003,0.238217,0.312232
3,0.567134,0.977871,-0.912960,-0.411432,-0.294397,0.504313,0.539069,-0.896969,-0.809829,-0.830898,...,-0.284042,-0.422755,-0.042114,0.594500,-0.488459,-1.262999,0.866200,1.258905,-0.591139,-0.432153
4,0.533265,1.703150,-0.030781,0.819186,0.811650,0.637775,0.301626,-0.983185,0.071542,-0.963455,...,-0.940665,-0.244708,1.343899,-0.762354,-1.065509,0.575636,1.675546,1.423447,-1.048803,-1.016314
5,0.132512,-0.421503,1.313266,0.261584,-0.053946,0.489721,-0.052090,-0.441390,2.094006,-0.202939,...,-1.170932,-0.384775,0.501331,-1.539369,-1.131513,0.737991,-0.032882,-0.491636,-1.115604,-1.278880


In [9]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0    1250.323886
1    1210.917257
2    1097.369000
3    1146.390000
4    1139.906000
5    1138.655000
Name: Espírito Santo - Consumo de Cimento (t), dtype: float64

In [10]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Espírito Santo - IDH,Espírito Santo - PIB - Estadual,Espírito Santo - PIB - Construção Civil,Espírito Santo - PIB - Per Capita,Espírito Santo - PIB - Preços de Mercado,Espírito Santo - IDH Educacao,Espírito Santo - IDH Longevidade,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Espírito Santo - Desemprego,Espírito Santo - IDH Renda,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Espírito Santo - Produção de Cimento (t)
6,-0.482916,-1.834285,1.511369,0.339831,-0.373097,0.345678,-0.483438,1.16956,2.035192,0.985175,...,-1.248372,-0.895379,-1.344493,-1.85042,-1.390537,-0.86501,-1.592914,-0.82445,-1.154201,-1.12558


In [11]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

6    1135.022
Name: Espírito Santo - Consumo de Cimento (t), dtype: float64

In [12]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=500, verbose=1)
    # Método estocástico e learning rate=0.01
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(2048, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(512, activation='tanh'),
        tf.keras.layers.Dense(64, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=5000, 
                      batch_size=len(train_input), 
                      validation_data=(train_input.iloc[-1:], train_target.iloc[-1:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [13]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(10)]
    print(random_seeds)

    best_val_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, 0, seed)

        current_val_loss = min(history.history['val_loss'])
        print(f"val_loss: {current_val_loss}")

        if current_val_loss < best_val_loss:
            best_val_loss = current_val_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [14]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[1591686954, 3866392051, 1115576, 3234278277, 558017436, 3494161720, 3987497588, 1278225659, 3012476169, 2159490815]


Step: 0 ___________________________________________


2023-08-19 18:36:51.370354: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-19 18:36:51.370734: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Epoch 846: early stopping
val_loss: 4.029273986816406e-05
winner_seed: 1591686954


Step: 1 ___________________________________________
Epoch 1343: early stopping
val_loss: 2.2664666175842285e-05
winner_seed: 3866392051


Step: 2 ___________________________________________
Epoch 804: early stopping
val_loss: 0.00033527612686157227


Step: 3 ___________________________________________
Epoch 945: early stopping
val_loss: 0.34676456451416016


Step: 4 ___________________________________________
Epoch 816: early stopping
val_loss: 0.0002675652503967285


Step: 5 ___________________________________________
Epoch 1858: early stopping
val_loss: 7.88271427154541e-06
winner_seed: 3494161720


Step: 6 ___________________________________________
Epoch 973: early stopping
val_loss: 2.518296241760254e-06
winner_seed: 3987497588


Step: 7 ___________________________________________
Epoch 798: early stopping
val_loss: 0.0004935860633850098


Step: 8 ___________________________________________
Epoch 7

In [15]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/5000
1/1 [==============================] - 0s 475ms/step - loss: 1356533.3750 - val_loss: 1111956.6250
Epoch 2/5000
1/1 [==============================] - 0s 38ms/step - loss: 1224747.0000 - val_loss: 3275717.7500
Epoch 3/5000
1/1 [==============================] - 0s 39ms/step - loss: 710504.2500 - val_loss: 3412619.5000
Epoch 4/5000
1/1 [==============================] - 0s 38ms/step - loss: 1222854.1250 - val_loss: 1881172.2500
Epoch 5/5000
1/1 [==============================] - 0s 39ms/step - loss: 1049660.6250 - val_loss: 2083579.1250
Epoch 6/5000
1/1 [==============================] - 0s 39ms/step - loss: 1007870.8125 - val_loss: 1926383.0000
Epoch 7/5000
1/1 [==============================] - 0s 40ms/step - loss: 970652.0000 - val_loss: 1872773.2500
Epoch 8/5000
1/1 [==============================] - 0s 39ms/step - loss: 944414.8125 - val_loss: 1834888.1250
Epoch 9/5000
1/1 [==============================] - 0s 38ms/step - loss: 903143.6875 - val_loss: 1772706.0000
Epoc

1/1 [==============================] - 0s 34ms/step - loss: 86646.2891 - val_loss: 148866.6875
Epoch 76/5000
1/1 [==============================] - 0s 35ms/step - loss: 83971.3516 - val_loss: 143345.2031
Epoch 77/5000
1/1 [==============================] - 0s 35ms/step - loss: 81396.4375 - val_loss: 139458.6250
Epoch 78/5000
1/1 [==============================] - 0s 35ms/step - loss: 78918.1250 - val_loss: 135411.1094
Epoch 79/5000
1/1 [==============================] - 0s 34ms/step - loss: 77229.6719 - val_loss: 135784.6094
Epoch 80/5000
1/1 [==============================] - 0s 33ms/step - loss: 74304.6641 - val_loss: 126576.6484
Epoch 81/5000
1/1 [==============================] - 0s 36ms/step - loss: 72025.4219 - val_loss: 123726.9453
Epoch 82/5000
1/1 [==============================] - 0s 35ms/step - loss: 69958.8047 - val_loss: 121350.2578
Epoch 83/5000
1/1 [==============================] - 0s 34ms/step - loss: 67920.4688 - val_loss: 117428.5312
Epoch 84/5000
1/1 [==============

1/1 [==============================] - 0s 33ms/step - loss: 10067.7842 - val_loss: 13812.3174
Epoch 151/5000
1/1 [==============================] - 0s 36ms/step - loss: 9998.6025 - val_loss: 14455.3945
Epoch 152/5000
1/1 [==============================] - 0s 37ms/step - loss: 9731.4658 - val_loss: 13794.7773
Epoch 153/5000
1/1 [==============================] - 0s 35ms/step - loss: 9360.8545 - val_loss: 12586.7744
Epoch 154/5000
1/1 [==============================] - 0s 36ms/step - loss: 8857.3574 - val_loss: 11398.1689
Epoch 155/5000
1/1 [==============================] - 0s 39ms/step - loss: 9111.6670 - val_loss: 10631.8203
Epoch 156/5000
1/1 [==============================] - 0s 35ms/step - loss: 11399.3232 - val_loss: 6706.6143
Epoch 157/5000
1/1 [==============================] - 0s 37ms/step - loss: 8884.8838 - val_loss: 10968.2666
Epoch 158/5000
1/1 [==============================] - 0s 36ms/step - loss: 8551.7227 - val_loss: 9956.3955
Epoch 159/5000
1/1 [=======================

1/1 [==============================] - 0s 36ms/step - loss: 1893.1063 - val_loss: 863.5434
Epoch 227/5000
1/1 [==============================] - 0s 37ms/step - loss: 2821.8337 - val_loss: 1327.5135
Epoch 228/5000
1/1 [==============================] - 0s 38ms/step - loss: 2122.0139 - val_loss: 768.3156
Epoch 229/5000
1/1 [==============================] - 0s 40ms/step - loss: 1957.3717 - val_loss: 5160.0688
Epoch 230/5000
1/1 [==============================] - 0s 37ms/step - loss: 3968.1953 - val_loss: 1404.9960
Epoch 231/5000
1/1 [==============================] - 0s 36ms/step - loss: 2616.6973 - val_loss: 1039.2123
Epoch 232/5000
1/1 [==============================] - 0s 39ms/step - loss: 2930.9280 - val_loss: 2507.2317
Epoch 233/5000
1/1 [==============================] - 0s 34ms/step - loss: 3384.6953 - val_loss: 1364.9291
Epoch 234/5000
1/1 [==============================] - 0s 36ms/step - loss: 3083.7214 - val_loss: 1675.6990
Epoch 235/5000
1/1 [==============================] - 

1/1 [==============================] - 0s 35ms/step - loss: 1973.2213 - val_loss: 452.5488
Epoch 304/5000
1/1 [==============================] - 0s 34ms/step - loss: 1681.6766 - val_loss: 225.8908
Epoch 305/5000
1/1 [==============================] - 0s 35ms/step - loss: 1607.8121 - val_loss: 447.4836
Epoch 306/5000
1/1 [==============================] - 0s 36ms/step - loss: 1826.2780 - val_loss: 693.3502
Epoch 307/5000
1/1 [==============================] - 0s 34ms/step - loss: 2386.9438 - val_loss: 66.2380
Epoch 308/5000
1/1 [==============================] - 0s 34ms/step - loss: 2189.9675 - val_loss: 149.5993
Epoch 309/5000
1/1 [==============================] - 0s 35ms/step - loss: 2176.8406 - val_loss: 122.0550
Epoch 310/5000
1/1 [==============================] - 0s 35ms/step - loss: 2174.0955 - val_loss: 123.5946
Epoch 311/5000
1/1 [==============================] - 0s 35ms/step - loss: 2171.9949 - val_loss: 118.6642
Epoch 312/5000
1/1 [==============================] - 0s 36ms/

...

Epoch 616/5000
1/1 [==============================] - 0s 32ms/step - loss: 2107.5891 - val_loss: 0.5217
Epoch 617/5000
1/1 [==============================] - 0s 34ms/step - loss: 2105.8516 - val_loss: 0.1003
Epoch 618/5000
1/1 [==============================] - 0s 34ms/step - loss: 2105.5339 - val_loss: 0.1616
Epoch 619/5000
1/1 [==============================] - 0s 35ms/step - loss: 2106.0225 - val_loss: 1.4413
Epoch 620/5000
1/1 [==============================] - 0s 35ms/step - loss: 2104.2805 - val_loss: 0.6580
Epoch 621/5000
1/1 [==============================] - 0s 36ms/step - loss: 2104.0249 - val_loss: 0.7995
Epoch 622/5000
1/1 [==============================] - 0s 33ms/step - loss: 2103.8679 - val_loss: 0.7488
Epoch 623/5000
1/1 [==============================] - 0s 35ms/step - loss: 2103.7302 - val_loss: 0.7459
Epoch 624/5000
1/1 [==============================] - 0s 37ms/step - loss: 2107.6870 - val_loss: 0.4806
Epoch 625/5000
1/1 [==============================] - 0s 38ms/st

1/1 [==============================] - 0s 35ms/step - loss: 2131.2935 - val_loss: 7.8690
Epoch 695/5000
1/1 [==============================] - 0s 34ms/step - loss: 2111.6150 - val_loss: 0.3118
Epoch 696/5000
1/1 [==============================] - 0s 36ms/step - loss: 2107.3572 - val_loss: 3.8563
Epoch 697/5000
1/1 [==============================] - 0s 35ms/step - loss: 2053.0117 - val_loss: 3.1816
Epoch 698/5000
1/1 [==============================] - 0s 34ms/step - loss: 2107.2283 - val_loss: 0.2707
Epoch 699/5000
1/1 [==============================] - 0s 37ms/step - loss: 2104.3123 - val_loss: 1.8317
Epoch 700/5000
1/1 [==============================] - 0s 36ms/step - loss: 2103.9873 - val_loss: 3.2860
Epoch 701/5000
1/1 [==============================] - 0s 33ms/step - loss: 2105.6777 - val_loss: 0.6298
Epoch 702/5000
1/1 [==============================] - 0s 36ms/step - loss: 2103.7981 - val_loss: 1.6548
Epoch 703/5000
1/1 [==============================] - 0s 36ms/step - loss: 2103

Epoch 773/5000
1/1 [==============================] - 0s 37ms/step - loss: 2104.2280 - val_loss: 3.8233
Epoch 774/5000
1/1 [==============================] - 0s 39ms/step - loss: 2106.4805 - val_loss: 5.5356
Epoch 775/5000
1/1 [==============================] - 0s 36ms/step - loss: 2106.0750 - val_loss: 4.9467
Epoch 776/5000
1/1 [==============================] - 0s 37ms/step - loss: 2105.7988 - val_loss: 4.9310
Epoch 777/5000
1/1 [==============================] - 0s 38ms/step - loss: 2138.4028 - val_loss: 1.1645
Epoch 778/5000
1/1 [==============================] - 0s 37ms/step - loss: 2117.1057 - val_loss: 9.0418
Epoch 779/5000
1/1 [==============================] - 0s 35ms/step - loss: 2106.6509 - val_loss: 4.0210
Epoch 780/5000
1/1 [==============================] - 0s 35ms/step - loss: 2108.2693 - val_loss: 6.0652
Epoch 781/5000
1/1 [==============================] - 0s 35ms/step - loss: 2105.5342 - val_loss: 4.2564
Epoch 782/5000
1/1 [==============================] - 0s 36ms/st

1/1 [==============================] - 0s 34ms/step - loss: 2447.9966 - val_loss: 0.1411
Epoch 852/5000
1/1 [==============================] - 0s 36ms/step - loss: 2112.4678 - val_loss: 6.0154
Epoch 853/5000
1/1 [==============================] - 0s 33ms/step - loss: 2107.1628 - val_loss: 3.0728
Epoch 854/5000
1/1 [==============================] - 0s 34ms/step - loss: 2106.5061 - val_loss: 3.5708
Epoch 855/5000
1/1 [==============================] - 0s 36ms/step - loss: 1975.8569 - val_loss: 19.6275
Epoch 856/5000
1/1 [==============================] - 0s 35ms/step - loss: 2115.0662 - val_loss: 8.4399
Epoch 857/5000
1/1 [==============================] - 0s 35ms/step - loss: 1947.4326 - val_loss: 3.6866
Epoch 858/5000
1/1 [==============================] - 0s 39ms/step - loss: 1933.0204 - val_loss: 5.7537
Epoch 859/5000
1/1 [==============================] - 0s 37ms/step - loss: 1991.8500 - val_loss: 28.2901
Epoch 860/5000
1/1 [==============================] - 0s 37ms/step - loss: 21

Epoch 930/5000
1/1 [==============================] - 0s 35ms/step - loss: 2102.7725 - val_loss: 1.2863
Epoch 931/5000
1/1 [==============================] - 0s 35ms/step - loss: 2102.7524 - val_loss: 1.2717
Epoch 932/5000
1/1 [==============================] - 0s 36ms/step - loss: 2102.7341 - val_loss: 1.2574
Epoch 933/5000
1/1 [==============================] - 0s 35ms/step - loss: 2102.7170 - val_loss: 1.2435
Epoch 934/5000
1/1 [==============================] - 0s 36ms/step - loss: 2102.7012 - val_loss: 1.2296
Epoch 935/5000
1/1 [==============================] - 0s 37ms/step - loss: 2106.4211 - val_loss: 3.3015
Epoch 936/5000
1/1 [==============================] - 0s 34ms/step - loss: 2151.5933 - val_loss: 19.7836
Epoch 937/5000
1/1 [==============================] - 0s 33ms/step - loss: 2113.3782 - val_loss: 0.0921
Epoch 938/5000
1/1 [==============================] - 0s 34ms/step - loss: 2103.2542 - val_loss: 1.7442
Epoch 939/5000
1/1 [==============================] - 0s 38ms/s

In [16]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 72ms/step
[1135.022](test_target) - [[1139.7538]](prediction) = 4.7317841796875655


In [17]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [18]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Espírito Santo - IDH,Espírito Santo - PIB - Estadual,Espírito Santo - PIB - Construção Civil,Espírito Santo - PIB - Per Capita,Espírito Santo - PIB - Preços de Mercado,Espírito Santo - IDH Educacao,Espírito Santo - IDH Longevidade,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Espírito Santo - Desemprego,Espírito Santo - IDH Renda,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Espírito Santo - Produção de Cimento (t)
0,-1.412893,-0.668631,1.310032,1.383732,1.384744,-1.409913,-1.414142,1.165029,1.396038,1.335266,...,-1.411409,1.414213,1.411717,-1.108290,1.099455,0.252910,-1.371172,-0.058369,1.064643,1.177038
1,0.653541,-0.744897,-0.193646,-0.438954,-0.443650,0.609512,0.719418,0.111769,-0.502296,-0.264151,...,0.628613,-0.705740,-0.633120,-0.206624,0.220596,1.078546,0.985446,-1.194517,0.273844,0.090403
2,0.759352,1.413528,-1.116386,-0.944778,-0.941095,0.800401,0.694724,-1.276798,-0.893743,-1.071116,...,0.782796,-0.708472,-0.778598,1.314914,-1.320051,-1.331456,0.385726,1.252886,-1.338486,-1.267442


train_target:


,Espírito Santo - Consumo de Cimento (t)
0,1250.323886
1,1210.917257
2,1097.369000


test_input:


...

,Espírito Santo - IDH,Espírito Santo - PIB - Estadual,Espírito Santo - PIB - Construção Civil,Espírito Santo - PIB - Per Capita,Espírito Santo - PIB - Preços de Mercado,Espírito Santo - IDH Educacao,Espírito Santo - IDH Longevidade,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Espírito Santo - Desemprego,Espírito Santo - IDH Renda,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Espírito Santo - Produção de Cimento (t)
6,-0.482916,-1.834285,1.511369,0.339831,-0.373097,0.345678,-0.483438,1.16956,2.035192,0.985175,...,-1.248372,-0.895379,-1.344493,-1.85042,-1.390537,-0.86501,-1.592914,-0.82445,-1.154201,-1.12558


test_target:


,Espírito Santo - Consumo de Cimento (t)
6,1135.022


Epoch 968: early stopping
1/1 [==============================] - 0s 51ms/step
Error: 4.7317841796875655




[1151.1097412109375, 1117.418701171875, 1143.1517333984375, 1139.7537841796875]

In [19]:
display(targets)
display(predictions)

[1146.39, 1139.906, 1138.655, 1135.022]

[1151.1097412109375, 1117.418701171875, 1143.1517333984375, 1139.7537841796875]

In [20]:
mae = mean_absolute_error(predictions, targets)
mae

9.10888940429686

In [21]:
porcentage = mae/np.mean(targets)
porcentage

0.007990301174412094